In [1]:
from stanfordcorenlp import StanfordCoreNLP
import numpy as np

In [2]:

zh_model = StanfordCoreNLP(r'/home/zzzqi/ViLT/nlp/stanford-corenlp-full-2018-10-05')
s_zh = 'a display of flowers growing out and over the retaining wall in front of cottages on a cloudy day.'
dep_zh = zh_model.dependency_parse(s_zh)
print(dep_zh)

[('ROOT', 0, 2), ('det', 2, 1), ('case', 4, 3), ('nmod', 2, 4), ('acl', 2, 5), ('compound:prt', 5, 6), ('cc', 5, 7), ('case', 11, 8), ('det', 11, 9), ('amod', 11, 10), ('conj', 5, 11), ('case', 13, 12), ('nmod', 11, 13), ('case', 15, 14), ('nmod', 13, 15), ('case', 19, 16), ('det', 19, 17), ('amod', 19, 18), ('nmod', 11, 19), ('punct', 2, 20)]


In [3]:
'''查找根结点对应的索引'''
root_index=[]
for i in range(len(dep_zh)):
    if dep_zh[i][0]=='ROOT':
        root_index.append(i)

In [4]:
'''修改依存关系三元组'''
new_dep_outputs=[]
for i in range(len(dep_zh)):
    for index in root_index:
        if i+1>index:
            tag=index

    if dep_zh[i][0]=='ROOT':	
        dep_output=(dep_zh[i][0],dep_zh[i][1],dep_zh[i][2]+tag)
    else:
        dep_output = (dep_zh[i][0], dep_zh[i][1] + tag, dep_zh[i][2] + tag)
    new_dep_outputs.append(dep_output)

print(new_dep_outputs)

[('ROOT', 0, 2), ('det', 2, 1), ('case', 4, 3), ('nmod', 2, 4), ('acl', 2, 5), ('compound:prt', 5, 6), ('cc', 5, 7), ('case', 11, 8), ('det', 11, 9), ('amod', 11, 10), ('conj', 5, 11), ('case', 13, 12), ('nmod', 11, 13), ('case', 15, 14), ('nmod', 13, 15), ('case', 19, 16), ('det', 19, 17), ('amod', 19, 18), ('nmod', 11, 19), ('punct', 2, 20)]


In [5]:
head_list = []
tokens = zh_model.word_tokenize(s_zh)
print(tokens)
# 求解headlist
for i in range(len(tokens)):
    for dep_output in new_dep_outputs:
        if dep_output[-1] == i + 1:
            head_list.append(int(dep_output[1]))

print(head_list)

# 得出邻接矩阵
def head_to_adj(head,max_sent_len):
    ret = np.zeros((max_sent_len, max_sent_len), dtype=np.float32)
    for i in range(len(head)):
        j=head[i]
        if j!=0:
            ret[i,j-1]=1
            ret[j-1,i]=1

    return ret

result = head_to_adj(head_list, 20)
print(result.reshape(1,-1,400))

['a', 'display', 'of', 'flowers', 'growing', 'out', 'and', 'over', 'the', 'retaining', 'wall', 'in', 'front', 'of', 'cottages', 'on', 'a', 'cloudy', 'day', '.']
[2, 0, 4, 2, 2, 5, 5, 11, 11, 11, 5, 13, 11, 15, 13, 19, 19, 19, 11, 2]
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
   1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
   1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.